In [140]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import scipy.signal as sp

In [141]:
def getImagesFromDir(dir: str, ext: str = ".jpg"):
    original = []
    gray = []
    for file in os.listdir(dir):
        if not file.endswith(ext):
            continue
        image = cv2.imread(f"{dir}/{file}")
        grayImage = cv2.imread(f"{dir}/{file}", 0)
        original.append(image)
        gray.append(grayImage)
    return original, gray

# FOR TESTING
# I = cv2.imread("img_00.jpg", 0)
# print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in I]))

In [142]:
def displayImage(image, title: str = "Image"):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [143]:
# Super jank temp solution for convolving 1d filter over 2d image
def correlate2DImage1DMask(I, mask):
    l, w = len(I), len(I[0])
    F = np.zeros((l, w))

    for i, row in enumerate(I):
        for j, pixel in enumerate(row):
            pre = []
            post = []
            s = j - 1
            e = j + 1
            while s < 0:
                pre.append(0)
                s += 1
            while e > w - 1:
                post.append(0)
                e -= 1
            sub = pre
            sub.extend(row[s:e+1])
            sub.extend(post)
            res = 0
            for x, f in zip(sub, mask):
                res += x * f
            F[i][j] = res
    return F


In [144]:

# Test image
# image = np.array([[10, 10, 10], [10, 40, 10], [80, 80, 80]])
DIR = "testRedChair"
# DIR = "testOffice"
images, grays = getImagesFromDir(DIR)

In [145]:
filtered = []

# Thresh = 6
# filter1 = np.array([-1, 0, 1]) * 0.5
# for image in grays:
#     F = correlate2DImage1DMask(image, filter1)
#     filtered.append(F)

# Thresh ~ 10-15
boxFilter3x3 = np.ones((3, 3)) * (1/9)

# Thresh ~ 10-15
boxFilter5x5 = np.ones((5, 5)) * (1/25)

for img in grays:
    res = sp.correlate2d(img, boxFilter5x5, mode="same", boundary="fill", fillvalue=0)
    filtered.append(res)


In [146]:
thresh = 10
delta = 1
size = (len(images[0]), len(images[0][0]))
res = np.zeros(size)
prev = res
masks = []
for t, image in enumerate(filtered):
    if t == 0:
        prev = image
        continue
    d = (np.subtract(image, prev)) / delta
    mask = np.where(abs(d) >= thresh, 1, 0)
    masks.append(mask)
    # x = image * mask
    # displayImage(x)
    


In [147]:
for image, mask in zip(images[1:], masks):
    x = image.copy()
    x[np.where(mask == 1)] = (0, 0, 255)
    displayImage(x)

In [148]:
# fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
# video = cv2.VideoWriter('video.avi', fourcc, 1, (240, 320))
# maskedImages, _ = getImagesFromDir("saved")
# for img in maskedImages:
#     video.write(img)
# cv2.destroyAllWindows()
# video.release()